# Building a coffee machine energy system model 


*This example is designed for the beginners to understand the basics of MESSAGEix framework through an easy and simple example. For detailed and full representation of a model in MESSAGEix framework, please refer to tutorials available online*

**Pre-requisites**
- Have succesfully installed *MESSAGEix*.


For information on how to install *MESSAGEix*, please refer to [Installation page](https://docs.messageix.org/en/stable/getting_started.html) and for getting *MESSAGEix* tutorials, please follow the steps mentioned in [Tutorials](https://docs.messageix.org/en/stable/tutorials.html).

Please refer to the [user guidelines](https://github.com/iiasa/message_ix/blob/master/NOTICE.rst)
for additional information on using *MESSAGEix*, including the recommended citation and how to name new models.


## *A stylized reference energy system model for Coffee machine*

<img src="coffee_system.png">


# Let's start! 


## 1) Importing required packages
For this model `message_ix`, `ixmp` and `pandas` (for data processing) Python packages are required. For other models,maybe some more packages are required which can be scripted under this part. 

In [ ]:
import ixmp                    
import message_ix 


from message_ix.utils import make_df

The *MESSAGEix* model is built using the *ixmp* `Platform`. The `Platform` is your connection to a database for storing model input data and scenario results. For more information,refer to ix modelling platform documentation [here](https://docs.messageix.org/projects/ixmp/en/stable/index.html)

In [ ]:
 mp = ixmp.Platform(name = 'local')

#### Building a new, empty MESSAGEix scenario

We assign our `model` snd `Scenario` a name and then store the  `Scenario` instance to a variable. A `Scenario` instance will contain all the model input data and results.

Let's see a list of sets included in MESSAGEix framework. We will not use all sets in this example though. 

In [ ]:
model = 'cofee_energy_model' # model name
scenario = 'test' # scenario name 
scen = message_ix.Scenario(mp, model, scenario,  version= 'new', annotation="…")

We can check the version of scenario instance using following command, if multiple scenario instances are stored in database

In [ ]:
scen.version

We can also see what sets, 

## 2) Spatial & temporal detail


We start with assigining a node, in this case, we assume my_home and let's say we want to include 2020 & 2021 in our study.

In [ ]:
scen.add_set('node', 'my_office')
scen.add_set('year', [2022, 2023])

## 3) Declaring members of the sets

Let's see a list of sets included in MESSAGEix framework. We will not use all sets in this example though. 

In [ ]:
scen.set_list()

We start filling in the model's `commodities`, `levels`, `technologies`, and `modes` (i.e., modes of operation of technologies. In this case, we have only one technology,`coffee_maker`, and then information how coffee maker operates as shown in figure above.. This information defines how certain technologies operate. 

In [ ]:
scen.add_set('technology', 'coffee_maker')
scen.add_set('commodity', ['water', 'electr', 'coffee_beans', 'coffee_cup']) # all members can be added at once
scen.add_set('level', ['pipe', 'storage', 'grid', 'useful'])
scen.add_set('mode', 'normal')    # modes of operation of a technology

Let's check if the sets are correctly added to `commodity`

In [ ]:
scen.set('commodity')

## 2. Adding data to parameters
Let's see a list of parameters in MESSAGEix framework

In [ ]:
scen.par_list()

We can also filter out parameters using basic Python fucntionality such as emission related parameters can be filtered out as;

In [ ]:
[x for x in scen.par_list() if 'emission' in x]

### 2.1) Adding data to parameter `demand`
We can always see the indexes of a parameter and make a table with those indexes (plus "value" and "unit"). The method is `idx_names()`


In [ ]:
scen.idx_names("demand")

Let's add a Python dataframe based on these indexes

In [ ]:
df = make_df('demand',
            node = 'my_office',
            commodity = 'coffee_cup',
            level = 'useful',
            year = [2022, 2023],
            value =  [200, 250],    # cup of coffee per year
            unit = '-',
            time = 'year')

In [ ]:
df

Now, we add this dataframe to the scenario instance

In [ ]:
scen.add_par('demand', df)
scen.par('demand') # to check, if correctly added

Similar method for including an `output` dataframe

In [ ]:
scen.idx_names("output")

In [ ]:
df = make_df('output',
            node_loc = 'my_office' ,
            node_dest = 'my_office',
           technology = 'coffee_maker',
           commodity = 'coffee_cup',
           level = 'useful',
           year_vtg = [2022, 2023],   # once installed can be active only one period
           year_act = [2022, 2023],
           mode = 'normal',
           value = 0.9,
           unit = '-',
           time = 'year',
           time_dest = 'year',)

In [ ]:
df

In [ ]:
scen.add_par('output', df)

In [ ]:
# Seeing the content of a parameter
scen.par('output')

Committing data of this scenario

In [ ]:
scen.commit('my first commit')   # This message is optional

If we want to add more stuff to this particular scenario instance, we can use the following command

In [ ]:
# scen.check_out()

Time to solve our model!

In [ ]:
scen.solve()

Getting objective value of the model

In [ ]:
scen.var('OBJ')

Getting activity of coffee maker

In [ ]:
scen.var('ACT')

Note: remember to close the platform before openning in another session

## Close the connection to the database

When working with local HSQLDB database instances, you cannot connect to one database from multipe Jupyter notebooks (or processes) at the same time.

If you want to easily switch between notebooks with connections to the same `ixmp` database, you need to close the connection in one notebook before initializing the platform using `ixmp.Platform()` in another notebook.

After having closed the database connection, you can reopen it using
```
mp.open_db()
```

In [ ]:
mp.close_db()

## Congratulations! 

You have built and run your very first *MESSAGEix* model. Welcome to the community!

The next tutorials will introduce you to other features of the framework, including energy system constraints, emissions taxes, and other policy options.

# Homework 
 
1. Make sure, the script we did during session if it run correctly on your machine. Please let us know, in case of any issues. 

2. We have added only one `commodity` in the output.Try to add more atleast two more commodities in this model. 

*Hint: Start by adding `demand` of that `commodity` and then following same steps as we did for `coffee_cup`*.
